In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cnn-lstm-training-output/cnn_training_output/cnn_training_output/confusion_matrix.png
/kaggle/input/cnn-lstm-training-output/cnn_training_output/cnn_training_output/best_plasma_cnn_model.pth
/kaggle/input/cnn-lstm-training-output/cnn_training_output/cnn_training_output/training_history.png
/kaggle/input/cnn-lstm-training-output/cnn_training_output/cnn_training_output/roc_curve.png
/kaggle/input/cnn-lstm-training-output/snn_training_output/snn_training_output/snn_training_results.png
/kaggle/input/cnn-lstm-training-output/snn_training_output/snn_training_output/best_plasma_snn_model.pth
/kaggle/input/cnn-lstm-training-output/snn_training_output/snn_training_output/snn_training_metrics.pth
/kaggle/input/cnn-lstm-training-output/snn_training_output/snn_training_output/snn_training_metrics.json
/kaggle/input/cnn-lstm-training-output/lstm_training_output/lstm_training_output/lstm_training_results.png
/kaggle/input/cnn-lstm-training-output/lstm_training_output/lstm_training_out

In [2]:
!pip install snntorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.6/125.6 kB 4.1 MB/s eta 0:00:00


In [7]:
"""
Working SNN Architecture - Based on Progressive Testing Success
Uses Level 2 architecture (no temporal processing) for reliable performance
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
import snntorch as snn
from snntorch import surrogate
import numpy as np
from sklearn.metrics import classification_report, balanced_accuracy_score
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import time
import json
from pathlib import Path
import warnings
import random
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔧 Working SNN Architecture on: {device}")

class PlasmaLikeDataset(Dataset):
    """More realistic dataset simulating plasma features"""
    def __init__(self, num_samples=2000, input_size=512, anomaly_ratio=0.3):
        self.num_samples = num_samples
        self.input_size = input_size
        self.anomaly_ratio = anomaly_ratio
        
        print(f"🔄 Creating plasma-like dataset: {num_samples} samples, {anomaly_ratio:.1%} anomalies")
        self.data, self.labels = self._generate_plasma_like_data()
        
        # Verify class balance
        unique, counts = torch.unique(self.labels, return_counts=True)
        print(f"   Class distribution: {dict(zip(unique.tolist(), counts.tolist()))}")
        
    def _generate_plasma_like_data(self):
        """Generate data that simulates plasma turbulence features"""
        num_anomalies = int(self.num_samples * self.anomaly_ratio)
        num_normal = self.num_samples - num_anomalies
        
        # Normal plasma patterns (based on 13 plasma features from memory)
        normal_data = []
        for i in range(num_normal):
            # Simulate magnetic field components and derived features
            base_pattern = torch.randn(self.input_size) * 0.5
            
            # Add plasma-like correlations
            b_field_strength = torch.randn(1) * 0.3
            base_pattern[:64] += b_field_strength  # Magnetic field components
            base_pattern[64:128] += b_field_strength * 0.5  # Derived features
            
            normal_data.append(base_pattern)
        
        normal_data = torch.stack(normal_data)
        normal_labels = torch.zeros(num_normal, dtype=torch.long)
        
        # Anomalous plasma patterns
        anomaly_data = []
        for i in range(num_anomalies):
            base_pattern = torch.randn(self.input_size) * 0.5
            
            # Add clear anomaly signatures
            anomaly_strength = 2.0 + np.random.random() * 1.0
            
            if i % 3 == 0:
                # Magnetic field spike anomalies
                base_pattern[:64] += torch.randn(64) * anomaly_strength
            elif i % 3 == 1:
                # Energy density anomalies
                base_pattern[128:256] += torch.ones(128) * anomaly_strength
            else:
                # Turbulence spectrum anomalies
                base_pattern[256:] += torch.sin(torch.linspace(0, 10, self.input_size-256)) * anomaly_strength
            
            anomaly_data.append(base_pattern)
        
        anomaly_data = torch.stack(anomaly_data)
        anomaly_labels = torch.ones(num_anomalies, dtype=torch.long)
        
        # Combine and shuffle
        data = torch.cat([normal_data, anomaly_data], dim=0)
        labels = torch.cat([normal_labels, anomaly_labels], dim=0)
        
        indices = torch.randperm(len(data))
        data = data[indices]
        labels = labels[indices]
        
        print(f"   Normal patterns: mean={normal_data.mean():.3f}, std={normal_data.std():.3f}")
        print(f"   Anomaly patterns: mean={anomaly_data.mean():.3f}, std={anomaly_data.std():.3f}")
        
        return data, labels
    
    def __len__(self):
        return self.num_samples
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]
    
    def get_balanced_sampler(self):
        """Get balanced sampler for training"""
        class_counts = torch.bincount(self.labels)
        class_weights = 1.0 / class_counts.float()
        sample_weights = class_weights[self.labels]
        return WeightedRandomSampler(sample_weights, len(sample_weights))

class WorkingSNN(nn.Module):
    """Working SNN based on successful Level 2 architecture"""
    def __init__(self, input_size=512, hidden_size=256, output_size=2):
        super(WorkingSNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        # Multi-layer architecture without temporal processing
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size // 2)
        self.fc3 = nn.Linear(hidden_size // 2, output_size)
        
        # LIF neurons with conservative parameters
        self.lif1 = snn.Leaky(beta=0.8, spike_grad=surrogate.straight_through_estimator())
        self.lif2 = snn.Leaky(beta=0.8, spike_grad=surrogate.straight_through_estimator())
        self.lif3 = snn.Leaky(beta=0.8, spike_grad=surrogate.straight_through_estimator(), output=True)
        
        # Conservative initialization
        for layer in [self.fc1, self.fc2, self.fc3]:
            nn.init.normal_(layer.weight, mean=0.0, std=0.01)
            nn.init.zeros_(layer.bias)
        
        # Light regularization
        self.dropout = nn.Dropout(0.1)
        
        print(f"✅ WorkingSNN: {input_size}→{hidden_size}→{hidden_size//2}→{output_size}")
    
    def forward(self, x):
        """Single-step forward pass (no temporal processing)"""
        # Fresh membrane potentials for each forward pass
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        mem3 = self.lif3.init_leaky()
        
        # Forward through network
        cur1 = self.fc1(x)
        spk1, mem1 = self.lif1(cur1, mem1)
        spk1 = self.dropout(spk1)
        
        cur2 = self.fc2(spk1)
        spk2, mem2 = self.lif2(cur2, mem2)
        spk2 = self.dropout(spk2)
        
        cur3 = self.fc3(spk2)
        spk3, mem3 = self.lif3(cur3, mem3)
        
        # Use membrane potential for classification
        return mem3

class WorkingSNNTrainer:
    """Trainer optimized for working SNN architecture"""
    def __init__(self, model, device):
        self.model = model.to(device)
        self.device = device
        
        # Balanced loss function
        self.criterion = nn.CrossEntropyLoss()
        
        # Conservative optimizer
        self.optimizer = torch.optim.AdamW(
            self.model.parameters(), 
            lr=0.001, 
            weight_decay=1e-5
        )
        
        self.scheduler = torch.optim.lr_scheduler.StepLR(
            self.optimizer, step_size=5, gamma=0.8
        )
        
    def train_epoch(self, dataloader):
        """Training epoch"""
        self.model.train()
        
        total_loss = 0
        correct = 0
        total = 0
        class_correct = [0, 0]
        class_total = [0, 0]
        
        for batch_idx, (data, labels) in enumerate(dataloader):
            data, labels = data.to(self.device), labels.to(self.device)
            
            self.optimizer.zero_grad()
            outputs = self.model(data)
            loss = self.criterion(outputs, labels)
            loss.backward()
            
            # Conservative gradient clipping
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            self.optimizer.step()
            
            # Metrics
            total_loss += loss.item()
            predicted = outputs.argmax(dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            for i in range(2):
                class_mask = labels == i
                if class_mask.sum() > 0:
                    class_correct[i] += ((predicted == labels) & class_mask).sum().item()
                    class_total[i] += class_mask.sum().item()
            
            if batch_idx % 20 == 0:
                class_acc_0 = class_correct[0] / max(class_total[0], 1) * 100
                class_acc_1 = class_correct[1] / max(class_total[1], 1) * 100
                balanced_acc = (class_acc_0 + class_acc_1) / 2
                
                print(f'  Batch {batch_idx}: Loss: {loss.item():.4f}, '
                      f'Balanced: {balanced_acc:.1f}% (N:{class_acc_0:.1f}%, A:{class_acc_1:.1f}%)')
        
        return total_loss / len(dataloader), 100. * correct / total, class_correct, class_total
    
    def validate(self, dataloader):
        """Validation"""
        self.model.eval()
        
        total_loss = 0
        correct = 0
        total = 0
        class_correct = [0, 0]
        class_total = [0, 0]
        
        all_predictions = []
        all_labels = []
        
        with torch.no_grad():
            for data, labels in dataloader:
                data, labels = data.to(self.device), labels.to(self.device)
                
                outputs = self.model(data)
                loss = self.criterion(outputs, labels)
                
                total_loss += loss.item()
                predicted = outputs.argmax(dim=1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                
                all_predictions.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                
                for i in range(2):
                    class_mask = labels == i
                    if class_mask.sum() > 0:
                        class_correct[i] += ((predicted == labels) & class_mask).sum().item()
                        class_total[i] += class_mask.sum().item()
        
        return total_loss / len(dataloader), 100. * correct / total, class_correct, class_total, all_predictions, all_labels

def main():
    """Working SNN training for plasma anomaly detection"""
    print("🚀 WORKING SNN FOR PLASMA ANOMALY DETECTION")
    print("=" * 60)
    print("🧠 Based on progressive testing: Level 2 architecture")
    print("🎯 Target: Stable balanced performance for plasma data")
    print("=" * 60)
    
    output_dir = Path("working_snn_output")
    output_dir.mkdir(exist_ok=True)
    
    # Create plasma-like datasets
    print("\n📊 Creating plasma-like datasets...")
    train_dataset = PlasmaLikeDataset(num_samples=4000, input_size=512, anomaly_ratio=0.3)
    val_dataset = PlasmaLikeDataset(num_samples=1000, input_size=512, anomaly_ratio=0.3)
    test_dataset = PlasmaLikeDataset(num_samples=800, input_size=512, anomaly_ratio=0.3)
    
    # Balanced sampling
    train_sampler = train_dataset.get_balanced_sampler()
    
    train_loader = DataLoader(train_dataset, batch_size=32, sampler=train_sampler, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)
    
    # Initialize working SNN
    print("\n🔧 Initializing working SNN...")
    model = WorkingSNN(input_size=512, hidden_size=256, output_size=2)
    trainer = WorkingSNNTrainer(model, device)
    
    # Training loop
    num_epochs = 15
    best_balanced_acc = 0
    patience = 5
    patience_counter = 0
    
    print(f"\n🧠 Training working SNN for {num_epochs} epochs...")
    
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 40)
        
        # Training
        train_loss, train_acc, train_class_correct, train_class_total = trainer.train_epoch(train_loader)
        
        # Validation
        val_loss, val_acc, val_class_correct, val_class_total, val_preds, val_labels = trainer.validate(val_loader)
        
        # Calculate metrics
        train_normal_acc = train_class_correct[0] / max(train_class_total[0], 1) * 100
        train_anomaly_acc = train_class_correct[1] / max(train_class_total[1], 1) * 100
        train_balanced_acc = (train_normal_acc + train_anomaly_acc) / 2
        
        val_normal_acc = val_class_correct[0] / max(val_class_total[0], 1) * 100
        val_anomaly_acc = val_class_correct[1] / max(val_class_total[1], 1) * 100
        val_balanced_acc = (val_normal_acc + val_anomaly_acc) / 2
        
        print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.1f}%, Balanced: {train_balanced_acc:.1f}%")
        print(f"       Normal: {train_normal_acc:.1f}%, Anomaly: {train_anomaly_acc:.1f}%")
        print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.1f}%, Balanced: {val_balanced_acc:.1f}%")
        print(f"       Normal: {val_normal_acc:.1f}%, Anomaly: {val_anomaly_acc:.1f}%")
        
        # Learning rate scheduling
        trainer.scheduler.step()
        
        # Early stopping
        if val_balanced_acc > best_balanced_acc:
            best_balanced_acc = val_balanced_acc
            patience_counter = 0
            torch.save(model.state_dict(), output_dir / "working_snn_model.pth")
            print(f"✅ New best model saved! Balanced Acc: {val_balanced_acc:.1f}%")
        else:
            patience_counter += 1
            
        if patience_counter >= patience:
            print(f"⏰ Early stopping after {patience} epochs without improvement")
            break
        
        # Check for collapse
        if val_normal_acc < 10 or val_anomaly_acc < 10:
            print(f"⚠️ Warning: Potential class collapse detected!")
    
    # Final evaluation
    print(f"\n📊 FINAL EVALUATION")
    print("=" * 30)
    
    # Load best model
    model.load_state_dict(torch.load(output_dir / "working_snn_model.pth", weights_only=True))
    
    # Test evaluation
    test_loss, test_acc, test_class_correct, test_class_total, test_preds, test_labels = trainer.validate(test_loader)
    
    test_normal_acc = test_class_correct[0] / max(test_class_total[0], 1) * 100
    test_anomaly_acc = test_class_correct[1] / max(test_class_total[1], 1) * 100
    test_balanced_acc = (test_normal_acc + test_anomaly_acc) / 2
    
    print(f"🎉 WORKING SNN TRAINING COMPLETE!")
    print(f"📊 Best Validation Balanced Accuracy: {best_balanced_acc:.1f}%")
    print(f"📊 Test Results:")
    print(f"   Overall Accuracy: {test_acc:.1f}%")
    print(f"   Balanced Accuracy: {test_balanced_acc:.1f}%")
    print(f"   Normal Accuracy: {test_normal_acc:.1f}%")
    print(f"   Anomaly Accuracy: {test_anomaly_acc:.1f}%")
    
    # Classification report
    print(f"\n📋 Classification Report:")
    print(classification_report(test_labels, test_preds, target_names=['Normal', 'Anomaly']))
    
    # Success analysis
    stability_check = (test_normal_acc > 30 and test_anomaly_acc > 30)
    performance_check = (test_balanced_acc > 70)
    
    print(f"\n🎯 WORKING SNN ANALYSIS:")
    print(f"Stability: {'✅ STABLE' if stability_check else '❌ UNSTABLE'} (Both classes >30%)")
    print(f"Performance: {'✅ GOOD' if performance_check else '⚠️ MODERATE'} (Balanced >{70 if not performance_check else test_balanced_acc:.0f}%)")
    
    if stability_check and performance_check:
        print("🎉 SUCCESS: Working SNN ready for integration!")
        recommendation = "Integrate working SNN into hybrid CNN-LSTM-SNN pipeline"
    elif stability_check:
        print("⚠️ PARTIAL SUCCESS: Stable but needs optimization")
        recommendation = "Optimize hyperparameters and architecture"
    else:
        print("❌ NEEDS WORK: Still showing instability")
        recommendation = "Consider simpler architectures or different approaches"
    
    # Save results
    results = {
        'best_val_balanced_accuracy': best_balanced_acc,
        'test_accuracy': test_acc,
        'test_balanced_accuracy': test_balanced_acc,
        'test_normal_accuracy': test_normal_acc,
        'test_anomaly_accuracy': test_anomaly_acc,
        'stability_achieved': stability_check,
        'performance_target_met': performance_check,
        'recommendation': recommendation,
        'ready_for_integration': stability_check and performance_check
    }
    
    with open(output_dir / "working_snn_results.json", 'w') as f:
        json.dump(results, f, indent=2)
    
    print(f"\n🎯 RECOMMENDATION: {recommendation}")
    print(f"📁 Results saved to: {output_dir}")
    
    return results

if __name__ == "__main__":
    torch.manual_seed(42)
    np.random.seed(42)
    random.seed(42)
    
    try:
        results = main()
        print("\n✅ Working SNN training completed!")
    except Exception as e:
        print(f"\n❌ Error during working SNN training: {e}")
        raise

🔧 Working SNN Architecture on: cuda
🚀 WORKING SNN FOR PLASMA ANOMALY DETECTION
🧠 Based on progressive testing: Level 2 architecture
🎯 Target: Stable balanced performance for plasma data

📊 Creating plasma-like datasets...
🔄 Creating plasma-like dataset: 4000 samples, 30.0% anomalies
   Normal patterns: mean=-0.001, std=0.514
   Anomaly patterns: mean=0.283, std=1.207
   Class distribution: {0: 2800, 1: 1200}
🔄 Creating plasma-like dataset: 1000 samples, 30.0% anomalies
   Normal patterns: mean=-0.007, std=0.511
   Anomaly patterns: mean=0.288, std=1.211
   Class distribution: {0: 700, 1: 300}
🔄 Creating plasma-like dataset: 800 samples, 30.0% anomalies
   Normal patterns: mean=-0.002, std=0.513
   Anomaly patterns: mean=0.276, std=1.205
   Class distribution: {0: 560, 1: 240}

🔧 Initializing working SNN...
✅ WorkingSNN: 512→256→128→2

🧠 Training working SNN for 15 epochs...

Epoch 1/15
----------------------------------------
  Batch 0: Loss: 0.6931, Balanced: 50.0% (N:100.0%, A:0.0%)
